<a href="https://colab.research.google.com/github/harjeet88/agentic_ai/blob/main/autogen/stock_picker_autogen_gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q autogen yfinance pandas openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 826.8/826.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.6 MB/s eta 0:00:00


In [4]:
!sudo apt-get update
!sudo apt-get install -y libta-lib-dev
#!pip install -q TA-Lib

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [74]:
#!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
#!tar -xzf ta-lib-0.4.0-src.tar.gz
#!cd ta-lib/ && ./configure --prefix=/usr && make && make install


In [6]:
!pip install ta-lib

  Using cached ta_lib-0.6.4.tar.gz (381 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for ta-lib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for ta-lib
Failed to build ta-lib
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (ta-lib)


In [45]:
import autogen
import os
import google.generativeai as genai
from dotenv import load_dotenv
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [55]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

In [56]:
GEMINI_API_KEY = GOOGLE_API_KEY

In [57]:
genai.configure(api_key=GOOGLE_API_KEY)
MODEL_NAME = "gemini-1.5-flash-latest" # Using the latest flash model
gemini=MODEL_NAME

In [58]:
def calculate_technical_indicators(ticker_symbol, period=200):
    """
    Calculates common technical indicators without TA-Lib.
    Args:
        ticker_symbol (str): The stock ticker symbol (e.g., "AAPL").
        period (int): The number of days for calculating moving averages.
    Returns:
        pandas.DataFrame: DataFrame with stock data and calculated indicators.
    """
    try:
        stock_data = yf.download(ticker_symbol, period=f"{period+60}d") # Fetch more data for indicators
        if stock_data.empty:
            print(f"Could not download data for {ticker_symbol}")
            return pd.DataFrame()

        df = stock_data.copy()

        # Moving Averages
        df['SMA_50'] = df['Close'].rolling(window=50).mean()
        df['SMA_200'] = df['Close'].rolling(window=200).mean()

        # RSI (Relative Strength Index) - Simple implementation
        delta = df['Close'].diff()
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
        avg_gain = gain.rolling(window=14).mean()
        avg_loss = loss.rolling(window=14).mean()
        rs = avg_gain / avg_loss
        df['RSI'] = 100 - (100 / (1 + rs))

        # MACD (Moving Average Convergence Divergence) - Simple implementation
        ema_slow = df['Close'].ewm(span=26, adjust=False).mean()
        ema_fast = df['Close'].ewm(span=12, adjust=False).mean()
        df['MACD'] = ema_fast - ema_slow
        df['Signal_Line'] = df['MACD'].ewm(span=9, adjust=False).mean()

        # Price Volume Action (Simple check for significant volume increase)
        df['Volume_Change'] = df['Volume'].pct_change() * 100
        df['Price_Change'] = df['Close'].pct_change() * 100

        # Filter for the last available data point
        return df.iloc[-1:]

    except Exception as e:
        print(f"Error calculating technical indicators for {ticker_symbol}: {e}")
        return pd.DataFrame()


In [59]:
# User Proxy Agent
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: "TERMINATE" in x.get("content", "").upper(),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={"work_dir": "coding"} # Ensure this directory exists
)

In [60]:
# Google LLM Agent for Fundamental Analysis
def create_fundamental_agent(config_list_gemini):
    return autogen.AssistantAgent(
        name="FundamentalAnalyst",
        system_message="""You are an expert fundamental stock analyst.
        Your task is to analyze a given stock's fundamentals and determine if it's a good investment.
        Focus on:
        1. Company's financial health (revenue growth, profit margins, debt levels, cash flow).
        2. Market position and competitive advantages.
        3. Management quality and strategy.
        4. Industry trends and outlook.
        5. Valuation (P/E, P/B, etc. if easily accessible or inferable).
        Use web search to gather information about the company and its industry.
        Provide a clear recommendation: 'INVEST' or 'DO NOT INVEST' based on your fundamental analysis.
        If investing, explain why the fundamentals support the investment.
        If not investing, explain the fundamental reasons.
        Do not perform technical analysis.
        Output your final decision in the format: FUNDAMENTAL_DECISION: [INVEST/DO NOT INVEST] FUNDAMENTAL_REASON: [Your explanation]""",
        llm_config={
        "cache_seed": 41,
        "config_list": config_list_gemini,
        "seed": 42
    },
        code_execution_config=False,
    )

In [61]:
# Google LLM Agent for Technical Analysis
def create_technical_agent(config_list_gemini):
    return autogen.AssistantAgent(
        name="TechnicalAnalyst",
        system_message="""You are an expert technical stock analyst.
        Your task is to analyze a given stock's technical indicators and provide a trading recommendation.
        You will be provided with the output of a Python function that calculates indicators like SMA (50, 200), RSI, MACD, and price/volume action.
        Analyze these indicators to suggest whether it's a good time to buy or sell based on technicals.
        Consider:
        - Moving Averages: Crossovers, support/resistance levels.
        - RSI: Overbought/oversold conditions.
        - MACD: Crossovers, divergence.
        - Price-Volume Action: Momentum, potential breakouts/breakdowns.
        Provide a clear recommendation: 'BUY', 'SELL', or 'HOLD' based on your technical analysis.
        If BUY or SELL, suggest potential entry/exit levels.
        Output your final decision in the format: TECHNICAL_DECISION: [BUY/SELL/HOLD] TECHNICAL_REASON: [Your explanation]""",
        #llm_config=llm_config,
        llm_config={
        "cache_seed": 41,
        "config_list": config_list_gemini,
        "seed": 42
    },
        code_execution_config={"work_dir": "coding"}, # Agent needs to execute Python code
    )


In [69]:
# Google LLM Agent for Investment Strategy
strategy_agent="""You are an investment strategist.
        You will receive inputs from the FundamentalAnalyst and TechnicalAnalyst.
        Your role is to synthesize these analyses and provide a final investment recommendation and potential entry/exit levels.
        If both FundamentalAnalyst and TechnicalAnalyst recommend 'INVEST' or 'BUY' respectively:
            - Confirm if the stock is a good overall investment.
            - Suggest a specific entry level (e.g., "enter around $X if price dips to Y").
            - Suggest an exit level (e.g., "consider exiting around $Z on target reached or stop-loss at $W").
            - Base entry/exit on both fundamental strength and technical signals.
        If either analysis is negative, explain why the combined recommendation is to 'DO NOT INVEST' or 'AVOID'.
        Your final output should be a clear investment strategy.
        Output your final strategy in the format: INVESTMENT_STRATEGY: [RECOMMENDATION] ENTRY_LEVEL: [Level/Condition] EXIT_LEVEL: [Level/Condition] STRATEGY_REASON: [Explanation]"""

def create_strategy_agent(config_list_gemini):
    return autogen.AssistantAgent(
        name="InvestmentStrategist",
        system_message=strategy_agent,
        #llm_config=llm_config,
        llm_config={
        "cache_seed": 41,
        "config_list": config_list_gemini,
        "seed": 42
    },
        code_execution_config=False,
    )

In [70]:
def run_stock_analysis(stock_ticker):
    """
    Runs the full stock analysis pipeline using AutoGen agents.
    """
    print(f"--- Starting Stock Analysis for {stock_ticker} ---")

    # LLM Configuration for all assistant agents
    llm_config = [
    {
        "model": gemini,
        "api_key": GOOGLE_API_KEY,
        "api_type": "google"
    }
]
    lm_config={
        "cache_seed": 41,
        "config_list": llm_config,
        "seed": 42
    }


    # Create agents
    fundamental_analyst = create_fundamental_agent(llm_config)
    technical_analyst = create_technical_agent(llm_config)
    investment_strategist = create_strategy_agent(llm_config)

    # Define the group chat
    groupchat = autogen.GroupChat(
        agents=[user_proxy, fundamental_analyst, technical_analyst, investment_strategist],
        messages=[],
        max_round=20, # Increased max rounds to allow for more complex interactions
    )
    manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=lm_config)

    # Start the conversation
    user_proxy.initiate_chat(
        manager,
        message=f"""Analyze the stock ticker: {stock_ticker}.
        First, perform fundamental analysis to assess if it's a worthwhile investment.
        Second, perform technical analysis to identify optimal entry and exit points if fundamentally sound.
        Finally, provide an overall investment strategy."""
    )

    print(f"--- Stock Analysis for {stock_ticker} Complete ---")


In [73]:
ticker_to_analyze="CDSL.NS"
if not ticker_to_analyze:
  print("No ticker symbol entered. Exiting.")
else:
  # Check if technical data can be fetched first
  print(f"Fetching initial data for {ticker_to_analyze} to check for technical indicator availability...")
  initial_data_check = yf.download(ticker_to_analyze, period="1mo") # Download recent data
  if initial_data_check.empty:
    print(f"Error: Could not download any data for ticker {ticker_to_analyze}. Please check the ticker symbol or try again later.")
  else:
    # If data is available, proceed with the AutoGen analysis
    run_stock_analysis(ticker_to_analyze)

/tmp/ipython-input-73-2376159190.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  initial_data_check = yf.download(ticker_to_analyze, period="1mo") # Download recent data
[*********************100%***********************]  1 of 1 completed

Fetching initial data for CDSL.NS to check for technical indicator availability...
--- Starting Stock Analysis for CDSL.NS ---


user_proxy (to chat_manager):

Analyze the stock ticker: CDSL.NS.
        First, perform fundamental analysis to assess if it's a worthwhile investment.
        Second, perform technical analysis to identify optimal entry and exit points if fundamentally sound.
        Finally, provide an overall investment strategy.

--------------------------------------------------------------------------------

Next speaker: FundamentalAnalyst

FundamentalAnalyst (to chat_manager):

I will perform only the fundamental analysis as requested.  I cannot provide technical analysis.

**Fundamental Analysis of CDSL.NS (Central Depository Services (India) Limited):**

CDSL is India's second-largest depository, playing a crucial role in the country's burgeoning securities market.  Its fundamentals are generally strong, but a nuanced assessment is needed.

**1. Company's Financial Health:**

* **Revenue Growth:** CDSL consistently demonstrates strong revenue growth, driven by increasing market participation

ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-1.5-flash'}, 'quotaValue': '15'}]}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.RetryInfo', 'retryDelay': '14s'}]}}

In [71]:
# Get stock ticker from user input
ticker_to_analyze = input("Enter the stock ticker symbol (e.g., AAPL, MSFT): ").strip().upper()


Enter the stock ticker symbol (e.g., AAPL, MSFT): AAPL
